In [32]:
region = "us-east-1"
instance_type = "ml.m4.16xlarge"
instance_count = 1
role = "arn:aws:iam::734840029783:role/SMStudioExecutionRole"
script_path = "s3://sagemaker-samdwar-iad/casework/nearmap/fsx/script.py"
fsx_id = "fs-0388bcb2a09bd3b51"
subnet_id = ["subnet-06dfe4259b8f518a3"]
sg_id = ["sg-519f1774"]
training_image = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.12.1-cpu-py38-ubuntu20.04-sagemaker"

In [33]:
import sagemaker


In [34]:
import boto3

session = boto3.session.Session(region_name=region)
aws_region = session.region_name
print(aws_region)

us-east-1


In [40]:
from sagemaker.inputs import FileSystemInput

file_system_directory_path = '/fsx'
file_system_access_mode = 'ro'

# Specify your file system type.
file_system_type = 'FSxLustre'

train = FileSystemInput(file_system_id=fsx_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

data_channels = {'fsx': train}

In [41]:
from sagemaker.pytorch.estimator import PyTorch

sagemaker_session = sagemaker.session.Session(boto_session=session)

estimator = PyTorch(
    image_uri=training_image,
    role=role,
    py_version='py3',
    instance_count=instance_count, 
    instance_type=instance_type,
    entry_point="script.py",
    volume_size = 100,
    max_run = 400000,
    output_path="s3://sagemaker-samdwar-iad/casework/nearmap/fsx/output",
    sagemaker_session=sagemaker_session, 
    #hyperparameters = hyperparameters,
    subnets=subnet_id,
    security_group_ids=sg_id
)

In [42]:
estimator.fit(inputs=data_channels, job_name="nearmap-fsx-ml-m4-16xlarge-002", wait=True)

2022-11-02 08:51:53 Starting - Starting the training job...
2022-11-02 08:52:21 Starting - Preparing the instances for trainingProfilerReport-1667379112: InProgress
.........
2022-11-02 08:53:58 Downloading - Downloading input data...
2022-11-02 08:54:38 Training - Downloading the training image...
2022-11-02 08:55:13 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-11-02 08:55:15,823 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-11-02 08:55:15,826 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-02 08:55:15,835 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-11-02 08:55:15,843 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-11-02 08:55:47,898 sagemaker-training-toolkit INFO     No G

Job ended with status 'Stopped' rather than 'Completed'. This could mean the job timed out or stopped early for some other reason: Consider checking whether it completed as you expect.


Training seconds: 880
Billable seconds: 880
